In [1]:
import torch
import torch.nn as nn
from IPython.display import clear_output
from IPython.core.debugger import set_trace
from torch.nn import functional as F
from torch import optim
import math

%load_ext autoreload
%autoreload 2
import dlc_practical_prologue as dl

In [8]:
train_input, train_target, test_input, test_target = dl.load_data(flatten=False)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [9]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.block1 = nn.Sequential(nn.Conv2d(1,32, kernel_size=5,stride=1),
                                    nn.MaxPool2d(kernel_size=2,stride=2),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU())
        self.block2 = nn.Sequential(nn.Conv2d(32,16, kernel_size=5,stride=1),
                                    nn.MaxPool2d(kernel_size=2,stride=2),
                                    nn.BatchNorm2d(16),
                                    nn.ReLU())
        self.lins = nn.Sequential(nn.Linear(256, 84),
                                  nn.ReLU(),
                                  nn.Linear(84, 10))

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.lins(x.view(-1,256))
        return x

In [10]:
class Net2(nn.Module):
    def __init__(self,n_hidden = 100,chan = 1):
        super(Net2,self).__init__()
        self.hidden = n_hidden
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(chan,32,kernel_size=3),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3),
            nn.MaxPool2d(kernel_size=2,stride=2)
            ,nn.BatchNorm2d(64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(1600,n_hidden),
           # nn.Dropout(0.5),
            nn.Linear(n_hidden,10)
            #nn.Softmax2d()
        )
    def forward(self,x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x.view(x.size(0),-1))
        return x

In [11]:
def train_model(model,train_input,train_target,nb_epochs=5):
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    batch_size = 100
    criterion = nn.CrossEntropyLoss()
    for e in range(nb_epochs):
        clear_output(wait=True)
        print("Progression:{:.2f}".format(e/nb_epochs*100))
        for inputs,targets in zip(train_input.split(batch_size),
                            train_target.split(batch_size)):
            output = model(inputs)
            loss = criterion(output,targets)
            model.zero_grad()
            loss.backward()
            optimizer.step()

In [32]:
def get_accuracy(model,inputs,targets):
    assert(inputs.size(0) == targets.size(0))
    tot_loss = 0
    nb_correct = 0
    batch_size = 20
    for train,target in zip(inputs.split(batch_size),
                           targets.split(batch_size)):
        pred = model(train)
        pred = torch.argmax(pred,axis = 1)
        nb_correct += (pred == target).int().sum().item()
    accuracy = nb_correct /inputs.size(0)
    print("accuracy: %.2f" % (accuracy) )
    return accuracy

In [40]:
def Kfold_CV(classtype,inputs,targets,K=4):
    assert(K>=2)
    N = inputs.size(0)
    indxes = torch.randperm(N)\
                  .split(int(N/K))
    accs = torch.empty(K)
    for k in range(K):
        model = classtype()
        
        test_indx = indxes[k]
        train_indx = torch.cat((indxes[:k]+indxes[k+1:]),0)
        
        train_inp,train_targ = inputs[train_indx],targets[train_indx]
        test_inp,test_targ = inputs[test_indx],targets[test_indx]
        train_model(model,train_inp,train_targ)
        acc = get_accuracy(model,train_inp,train_targ)
        accs[k] = acc
    print("Accuracies for {}-fold:{}".format(K,accs.tolist()))
    print("Mean acc:{}".format(accs.mean()))

In [41]:
Kfold_CV(Net2,train_input,train_target)

Progression:80.00
accuracy: 0.99
Accuracies for 4-fold:[0.9973333477973938, 0.9946666955947876, 0.9946666955947876, 0.9919999837875366]
Mean acc:0.9946666955947876


In [35]:
model = Net2()
train_model(model,train_input[indxes][:500],train_target[indxes][:500])
get_accuracy(model,train_input[indxes][:500],train_target[indxes][:500])

Progression:80.00
accuracy: 0.99


0.994

In [24]:
N = train_input.size(0)
indxes = torch.randperm(N)\
              .split(int(N/2))
model = Net2()

test_indx = indxes[0]
train_indx = torch.cat((indxes[:0]+indxes[1:]),0)

train_inp,train_targ = train_input[train_indx],train_target[train_indx]
test_inp,test_targ = train_input[test_indx],train_target[test_indx]
train_model(model,train_inp,train_targ)
acc = get_accuracy(model,train_inp,train_targ)

Progression:80.00
accuracy: 0.50


In [27]:
train_indx

tensor([155, 911, 960, 199, 427, 495, 201, 348, 722, 659, 927, 132, 547, 161,
        235, 126, 666,  93, 642, 839, 553, 479, 335, 108, 231,   8, 836, 539,
        650, 210, 329, 801, 660, 891, 871, 804, 380, 800, 756, 179, 386, 670,
        175,  95, 906, 892, 658, 250, 417, 959, 887, 399, 454, 292,  56, 783,
        619, 261, 167, 187, 274, 381, 226,  53, 686, 878, 957, 573, 398, 359,
        664, 423, 343, 815, 655,  13, 312, 603, 415, 103, 834, 770, 758, 995,
        419, 773,  91, 618,  27, 987,  52, 797, 868, 727, 104, 296, 925, 273,
        934, 537, 703, 968, 476, 182, 530, 828, 151, 825, 450, 416, 723, 654,
        326, 761, 357,   2,  61, 331, 969,  96, 137, 146, 394, 982, 321, 521,
        180, 506, 185, 683, 421, 452, 744, 605, 558, 685, 590, 220,  82, 557,
        156, 894, 129, 254, 926, 693, 318, 838, 159, 402, 572, 542, 289, 511,
        798, 984, 657, 819, 516, 989, 431, 447, 567, 480, 458, 678, 485, 544,
        765, 725, 813, 653, 812, 378, 323, 581, 912, 656, 954,  